In [14]:
from physioex.train.utils import train, test, finetune
from physioex.data import PhysioExDataModule

from physioex.train.networks.utils.loss import config as loss_config
from physioex.train.networks import config as network_config

datamodule = PhysioExDataModule(
    datasets=["sleepedf"],     # list of datasets to be used
    batch_size=64,             # batch size for the DataLoader
    preprocessing="raw",       # preprocessing method
    selected_channels=["EEG"], # channels to be selected
    sequence_length=21,        # length of the sequence
    data_folder="./data",      # path to the data folder
)

# get the DataLoaders
train_loader = datamodule.train_dataloader()
val_loader = datamodule.val_dataloader()
test_loader = datamodule.test_dataloader()

print(len(train_loader)) # number of batches in the training set
print(len(val_loader)) # number of batches in the validation set
print(len(test_loader)) # number of batches in the test set

checkpoint_path = "./model/checkpoint/"

2036
570
409


In [15]:
import os
import yaml
import importlib

# with open("./physioex/physioex/train/networks/config/chambon2018.yaml", "r") as file:
with open("./config.yaml", "r") as file:
    config = yaml.safe_load(file)

print(config)
networks_config = config["model_config"]

print(networks_config)

# load the loss function 
loss_package, loss_class = networks_config["loss_call"].split(":")
model_loss = getattr(importlib.import_module(loss_package), loss_class)

print(model_loss)

# in case you provide model_name the system loads the additional model parameters from the library
if "model_name" in config:
    model_name = config["model_name"]

print(model_name)

# load the model class
model_package, model_class = config["module"].split(":")
model_class = getattr(importlib.import_module(model_package), model_class)

print(model_class)

{'model_config': {'loss_call': 'physioex.train.networks.utils.loss:CrossEntropyLoss', 'loss_params': {}, 'seq_len': 21, 'n_classes': 5, 'n_channels': 1, 'in_channels': ['EEG'], 'sfreq': 100, 'n_times': 3000, 'learning_rate': '1e-4', 'adam_beta_1': 0.9, 'adam_beta_2': 0.999, 'adam_epsilon': '1e-8', 'latent_space_dim': 32, 'input_processing': 'raw'}, 'checkpoint_path': './model/checkpoint/', 'input_transform': 'raw', 'model': 'physioex.train.networks.chambon2018:Chambon2018Net', 'model_name': 'chambon2018', 'module': 'physioex.train.networks:Chambon2018Net', 'preprocessing': 'raw', 'target_transform': 'physioex.train.networks.utils.target_transform:get_mid_label'}
{'loss_call': 'physioex.train.networks.utils.loss:CrossEntropyLoss', 'loss_params': {}, 'seq_len': 21, 'n_classes': 5, 'n_channels': 1, 'in_channels': ['EEG'], 'sfreq': 100, 'n_times': 3000, 'learning_rate': '1e-4', 'adam_beta_1': 0.9, 'adam_beta_2': 0.999, 'adam_epsilon': '1e-8', 'latent_space_dim': 32, 'input_processing': 'ra

In [16]:
datamodule_kwargs = {
    "selected_channels" : ["EEG"], # needs to match in_channels
    "sequence_length" : networks_config["seq_len"],
    "target_transform" : config["target_transform"],
    "preprocessing" : config["input_transform"],
    "data_folder" : "./data",
}

# Train the model
best_checkpoint = train(
    datasets = datamodule,
    datamodule_kwargs = datamodule_kwargs,
    model = model_name,
    model_class = model_class,
    model_config = network_config,
    checkpoint_path = checkpoint_path,
    batch_size = 64,
    max_epochs = 10
)

# Test the model
results_dataframe = test(
    datasets = datamodule,
    datamodule_kwargs = datamodule_kwargs,
    model_class = model_class,
    model_config = network_config,
    chekcpoint_path = os.path.join( checkpoint_path, best_checkpoint ),
    batch_size = 64,
    results_dir = checkpoint_path,  # if you want to save the test results 
                                    # in your checkpoint directory
)

Seed set to 42


ValueError: You requested to find -1 devices but there are no visible CUDA devices on this machine.

In [ ]:
# check if mps available and set device to mps
